# Graph DB QA chain

This notebook shows how to use LLMs to provide a natural language interface to [HugeGraph](https://hugegraph.apache.org/cn/) database.

You will need to have a running HugeGraph instance.
You can run a local docker container by running the executing the following script:

```
docker run \
    --name=graph \
    -itd \
    -p 8080:8080 \
    hugegraph/hugegraph
```

If we want to connect HugeGraph in the application, we need to install python sdk:

```
pip3 install hugegraph-python
```

If you are using the docker container, you need to wait a couple of second for the database to start, and then we need create schema and write graph data for the database.

In [2]:
from hugegraph.connection import PyHugeGraph
client = PyHugeGraph("localhost", "8080", user="admin", pwd="admin", graph="langchain")

First, we create the schema for a simple movie database:

In [2]:
"""schema"""
schema = client.schema()
schema.propertyKey("name").asText().ifNotExist().create()
schema.propertyKey("birthDate").asText().ifNotExist().create()
schema.vertexLabel("Person").properties("name", "birthDate").usePrimaryKeyId().primaryKeys(
    "name").ifNotExist().create()
schema.vertexLabel("Movie").properties("name").usePrimaryKeyId().primaryKeys("name").ifNotExist().create()
schema.edgeLabel("ActedIn").sourceLabel("Person").targetLabel("Movie").ifNotExist().create()

Then we can insert some data.

In [2]:
"""graph"""
g = client.graph()
g.addVertex("Person", {"name": "Al Pacino", "birthDate": "1940-04-25"})
g.addVertex("Person", {"name": "Robert De Niro", "birthDate": "1943-08-17"})
g.addVertex("Movie", {"name": "The Godfather"})
g.addVertex("Movie", {"name": "The Godfather Part II"})
g.addVertex("Movie", {"name": "The Godfather Coda The Death of Michael Corleone"})

g.addEdge("ActedIn", "12:Al Pacino", "13:The Godfather", {})
g.addEdge("ActedIn", "12:Al Pacino", "13:The Godfather Part II", {})
g.addEdge("ActedIn", "12:Al Pacino", "13:The Godfather Coda The Death of Michael Corleone", {})
g.addEdge("ActedIn", "12:Robert De Niro", "13:The Godfather Part II", {})

## Creating `HugeGraphQAChain`

We can now create the `HugeGraph` and `HugeGraphQAChain`. To create the `HugeGraph` we simply need to pass the database object to the `HugeGraph` constructor.

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import HugeGraphQAChain
from langchain.graphs import HugeGraph

In [2]:
graph = HugeGraph(
    username="default",
    password="default",
    address="address",
    port=8081,
    graph="graph"
)

## Refresh graph schema information

If the schema of database changes, you can refresh the schema information needed to generate Gremlin statements.

In [4]:
# graph.refresh_schema()

In [2]:
print(graph.get_schema)

Node properties: [name: Movie, primary_keys: ['name'], properties: ['name'], name: Person, primary_keys: ['name'], properties: ['name', 'birthDate']]
Edge properties: [name: ActedIn, properties: []]
Relationships: ['Person--ActedIn-->Movie']



## Querying the graph

We can now use the graph Gremlin QA chain to ask question of the graph

In [6]:
chain = HugeGraphQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [7]:
chain.run("Who played in The Godfather?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
g.V().has("name", "The Godfather").in("ActedIn")
Full Context:
[{'id':'12:Al Pacino','label':'Person','properties':{'name':'Al Pacino','birthDate':'1940-04-25'}}]

> Finished chain.


'Al Pacino played in The Godfather.''